In [163]:
import numpy as np
from stl import mesh
from PIL import Image
import matplotlib.pyplot as plt
from PIL import ImageOps

In [164]:
def get_model_from_img(img: str, scale_reduction = 1, maxHeight = 100, invert = False):
    
    output_name = img[:-3] + 'stl'
    
    img = Image.open(img).convert('L')
    if invert:
        img = ImageOps.invert(img)
    img = ImageOps.mirror(img)
    plt.imshow(img)
    
    scale = scale_reduction
    max_size = (img.size[1] / scale, img.size[0] / scale)

    img.thumbnail(max_size)
    imageNP = np.array(img)
    print(img.size)

    nrows = img.size[1]
    ncols = img.size[0]

    vertices = np.zeros((nrows, ncols, 3))
    vertices_bottom = np.zeros((nrows, ncols, 3))
    faces = []

    minPix = imageNP.min()
    maxPix = imageNP.max()

    #Top
    for x in range(0, ncols):
        for y in range(0, nrows):
            pixelIntensive = imageNP[y][x]

            z = pixelIntensive * maxHeight / maxPix
            vertices[y][x] = (x, y, z)



    for x in range(0, ncols - 1):
        for y in range(0, nrows - 1):
            if vertices[y][x][2] != 0 or vertices[y][x+1][2] != 0 or vertices[y+1][x+1][2] !=0:

                face1 = (vertices[y][x], vertices[y][x+1], vertices[y+1][x+1])
                faces.append(face1)


                face2 = (vertices[y+1][x+1], vertices[y+1][x], vertices[y][x])
                faces.append(face2)



    #Bottom
    maxX = ncols-1
    maxY = nrows-1
    for x in range(0, ncols):
        for y in range(0, nrows):
            z = 0
            vertices_bottom[y][x] = (x, y, z)

    for x in range(0, ncols - 1):
        for y in range(0, nrows - 1):
            face1 = (vertices_bottom[y][x], vertices_bottom[y+1][x], vertices_bottom[y+1][x+1])
            faces.append(face1)


            face2 = (vertices_bottom[y+1][x+1], vertices_bottom[y][x+1], vertices_bottom[y][x])
            faces.append(face2)



    #Sides
    for x in range(0, ncols ):
        for y in range(0, nrows):
            if x == 0 and y != nrows - 1:
                    face1 = (vertices_bottom[y][x], vertices[y][x], vertices_bottom[y+1][x])
                    faces.append(face1)


                    face2 = (vertices_bottom[y+1][x], vertices[y][x], vertices[y+1][x])
                    faces.append(face2)

            if x == ncols - 1 and y != nrows - 1:
                    face1 = (vertices_bottom[y][x], vertices[y][x], vertices_bottom[y+1][x])
                    faces.append(face1)


                    face2 = (vertices_bottom[y+1][x], vertices[y][x], vertices[y+1][x])
                    faces.append(face2)
            if y == nrows - 1 and x != ncols - 1:
                    face1 = (vertices_bottom[y][x], vertices[y][x], vertices_bottom[y][x+1])
                    faces.append(face1)


                    face2 = (vertices_bottom[y][x+1], vertices[y][x], vertices[y][x+1])
                    faces.append(face2)

            if y == 0 and x != ncols - 1:
                    face1 = (vertices[y][x], vertices_bottom[y][x], vertices_bottom[y][x+1])
                    faces.append(face1)


                    face2 = (vertices_bottom[y][x+1], vertices[y][x+1], vertices[y][x])
                    faces.append(face2)


    facesNP = np.array(faces)

    surface = mesh.Mesh(np.zeros(facesNP.shape[0], dtype=mesh.Mesh.dtype))
    for i, f in enumerate(faces):
        for j in range(3):
            surface.vectors[i][j] = facesNP[i][j]

    surface.save(output_name)
    return print('Complete', output_name)
